In [12]:
import sqlite3
from faker import Faker
import random
import pandas as pd

cnx = sqlite3.connect('base_datos_proveedores.db')
cursor = cnx.cursor()



In [13]:
def sql_query(query):

    cursor.execute(query) 

    ans = cursor.fetchall()

    names = [description[0] for description in cursor.description]

    return pd.DataFrame(ans,columns=names)

In [14]:
df_operaciones = sql_query("SELECT * FROM operaciones as A left JOIN pieza as B on A.id_pieza = B.id")
df_operaciones

,id_operacion,id_proveedor,id_pieza,fecha,id_factura,id,nombre,codigo_pieza,color,precio,categoria
0,1170,106,49,2021-07-08,5022,49,Sensor de oxígeno,WS-78468,Negro,231.09,Refrigeración
1,1309,123,31,2024-04-29,5049,31,Llanta,OC-19410,Naranja,927.13,Frenos
2,1753,110,49,2023-03-03,5018,49,Sensor de oxígeno,WS-78468,Negro,231.09,Refrigeración
3,1899,112,46,2020-12-05,5006,46,Disco de freno,jE-73080,Marrón,458.28,Transmisión
4,1901,102,42,2023-11-05,5032,42,Batería,xA-14949,Negro,377.48,Iluminación
5,2108,122,34,2023-10-15,5017,34,Caja de cambios,yO-48161,Verde,214.21,Suspensión
6,2204,111,40,2021-06-12,5020,40,Faro delantero,Yp-80322,Blanco,375.77,Neumáticos
7,2330,110,53,2021-07-07,5007,53,Espejo retrovisor,cO-37145,Plateado,310.46,Iluminación
8,2367,115,33,2021-10-27,5027,33,Ventilador,Lu-81111,Gris,27.14,Escape
9,2473,113,40,2023-12-15,5024,40,Faro delantero,Yp-80322,Blanco,375.77,Neumáticos


In [15]:
df_operaciones.drop(columns=['id',"nombre","color","categoria", "codigo_pieza"], inplace=True)

In [16]:
df_operaciones

,id_operacion,id_proveedor,id_pieza,fecha,id_factura,precio
0,1170,106,49,2021-07-08,5022,231.09
1,1309,123,31,2024-04-29,5049,927.13
2,1753,110,49,2023-03-03,5018,231.09
3,1899,112,46,2020-12-05,5006,458.28
4,1901,102,42,2023-11-05,5032,377.48
5,2108,122,34,2023-10-15,5017,214.21
6,2204,111,40,2021-06-12,5020,375.77
7,2330,110,53,2021-07-07,5007,310.46
8,2367,115,33,2021-10-27,5027,27.14
9,2473,113,40,2023-12-15,5024,375.77


In [29]:
df_facturas = df_operaciones.copy()
print(type(df_facturas["fecha"]))
df_facturas["fecha"] = pd.to_datetime(df_facturas["fecha"])
df_facturas["fecha_factura"] = df_facturas["fecha"]
df_facturas["cantidad"] = 0
df_facturas["total"] = 0.0
for i in range(len(df_facturas)):
    df_facturas["fecha_factura"][i] = df_facturas["fecha"][i] + pd.DateOffset(days=(random.randint(1, 5)))
    df_facturas["cantidad"][i] = random.randint(1, 10)
    df_facturas["total"][i] = df_facturas["cantidad"][i] * df_facturas["precio"][i]

df_facturas


<class 'pandas.core.series.Series'>


C:\Users\luis_\AppData\Local\Temp\ipykernel_44800\824209675.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_facturas["fecha_factura"][i] = df_facturas["fecha"][i] + pd.DateOffset(days=(random.randint(1, 5)))
C:\Users\luis_\AppData\Loc

,id_operacion,id_proveedor,id_pieza,fecha,id_factura,precio,fecha_factura,cantidad,total
0,1170,106,49,2021-07-08,5022,231.09,2021-07-12,1,231.09
1,1309,123,31,2024-04-29,5049,927.13,2024-05-01,1,927.13
2,1753,110,49,2023-03-03,5018,231.09,2023-03-06,6,1386.54
3,1899,112,46,2020-12-05,5006,458.28,2020-12-10,4,1833.12
4,1901,102,42,2023-11-05,5032,377.48,2023-11-06,10,3774.80
5,2108,122,34,2023-10-15,5017,214.21,2023-10-17,3,642.63
6,2204,111,40,2021-06-12,5020,375.77,2021-06-16,10,3757.70
7,2330,110,53,2021-07-07,5007,310.46,2021-07-10,4,1241.84
8,2367,115,33,2021-10-27,5027,27.14,2021-10-28,4,108.56
9,2473,113,40,2023-12-15,5024,375.77,2023-12-18,4,1503.08


In [35]:
df_facturas.rename(columns={"precio":"precio_ud", "cantidad":"cantidad_piezas"}, inplace=True)
#df_facturas.drop(columns=["fecha"], inplace=True) ya se ejecuto
df_facturas = df_facturas.reindex(["id_factura","id_proveedor","id_pieza","cantidad_piezas","precio_ud","total","fecha_factura"],axis =1)
df_facturas

,id_factura,id_proveedor,id_pieza,cantidad_piezas,precio_ud,total,fecha_factura
0,5022,106,49,1,231.09,231.09,2021-07-12
1,5049,123,31,1,927.13,927.13,2024-05-01
2,5018,110,49,6,231.09,1386.54,2023-03-06
3,5006,112,46,4,458.28,1833.12,2020-12-10
4,5032,102,42,10,377.48,3774.80,2023-11-06
5,5017,122,34,3,214.21,642.63,2023-10-17
6,5020,111,40,10,375.77,3757.70,2021-06-16
7,5007,110,53,4,310.46,1241.84,2021-07-10
8,5027,115,33,4,27.14,108.56,2021-10-28
9,5024,113,40,4,375.77,1503.08,2023-12-18


In [36]:
df_facturas.to_sql('facturas', cnx, if_exists='append', index=False)

10